# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd

# Load data
df = pd.read_csv("homepage_actions.csv")

# Quick look
print(df.head())
print(df.info())

# Unique actions
print(df['action'].value_counts())

# How many unique users?
print("Total users:", df['id'].nunique())

# Any users in both groups?
dup_users = df.groupby('id')['group'].nunique()
print("Users in both groups:", sum(dup_users > 1))

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
action
view     6328
click    1860
Name: count, dtype: int64
Total users: 6328
Users in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [5]:
# Conduct a Statistical Test 

# Rebuild per-user dataset (in case Exploratory cell wasn't run)
user_df = df.pivot_table(index="id", columns="action", aggfunc="size", fill_value=0)

# Add group info
user_groups = df.drop_duplicates(subset=["id"])[["id", "group"]]
user_df = user_df.merge(user_groups, on="id", how="left")

# Recalculate clicks and views
clicks = user_df.groupby("group")["click"].sum()
views = user_df.groupby("group")["click"].count()

# Two-sample z-test for proportions
z, pval = proportions_ztest(count=clicks, nobs=views)

print("Clicks per group:\n", clicks)
print("Users per group:\n", views)
print("Z-statistic:", z)
print("P-value (statsmodels):", pval)

Clicks per group:
 group
control       932
experiment    928
Name: click, dtype: int64
Users per group:
 group
control       3332
experiment    2996
Name: click, dtype: int64
Z-statistic: -2.618563885349469
P-value (statsmodels): 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [6]:
# Expected clicks in experiment group if CTR was same as control
control_ctr = clicks['control'] / views['control']
expected_clicks_exp = control_ctr * views['experiment']

print("Expected clicks (if exp ~ control):", expected_clicks_exp)

Expected clicks (if exp ~ control): 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [7]:
# Variance of binomial distribution
variance = views['experiment'] * control_ctr * (1 - control_ctr)
std_dev = variance ** 0.5

actual_clicks_exp = clicks['experiment']
z_score = (actual_clicks_exp - expected_clicks_exp) / std_dev

print("Z-score (manual):", z_score)

Z-score (manual): 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [9]:
# Step 3: Manual P-Value

from scipy import stats   # make sure stats is imported

# Two-tailed p-value from z-score
p_val_manual = 2 * (1 - stats.norm.cdf(abs(z_score)))

print("Manual p-value:", p_val_manual)

Manual p-value: 0.0002497305601389943


### Analysis:

Does this result roughly match that of the previous statistical test?

> In this lab:  
- I conducted an exploratory analysis of homepage actions data.  
- I compared CTRs between control and experimental groups.  
- Using a two-sample proportion z-test, I found that (insert result depending on output).  
- I verified the results manually by calculating expected clicks, z-score, and p-value.  
- Both approaches matched closely, confirming the conclusion.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.